In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from sklearn.preprocessing import LabelEncoder


In [2]:
# 데이터 불러오기
df807 = pd.read_parquet('open/train/8.성과정보/201807_train_성과정보.parquet')
df808 = pd.read_parquet('open/train/8.성과정보/201808_train_성과정보.parquet')
df809 = pd.read_parquet('open/train/8.성과정보/201809_train_성과정보.parquet')
df810 = pd.read_parquet('open/train/8.성과정보/201810_train_성과정보.parquet')
df811 = pd.read_parquet('open/train/8.성과정보/201811_train_성과정보.parquet')
df812 = pd.read_parquet('open/train/8.성과정보/201812_train_성과정보.parquet')

df807.head()

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.0,0.409766,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,201807,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.0,0.0,-1.010817,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,201807,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.0,0.0,-0.902166,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,201807,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.0,0.0,-0.866052,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,201807,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN


In [3]:
# 8번 데이터 병합
df8 = pd.concat([df807, df808, df809, df810, df811], ignore_index=True)
df8 = df8.sort_values(by=['ID', '기준년월']).reset_index(drop=True)
df8

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.0,0.409766,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.0,1.044401,1.280543
1,201808,TRAIN_000000,-0.097118,-0.097118,-0.111664,-1.999996,0.0,0.0,0.0,0.192771,...,0.999998,0.595262,1.000661,0.999998,0.999998,-0.140747,-0.117855,0.0,1.185349,1.555697
2,201809,TRAIN_000000,0.015812,0.015812,0.015812,0.000000,0.0,0.0,0.0,0.250439,...,0.999998,1.048667,1.000722,0.999998,0.999998,-0.057002,0.052830,0.0,0.879170,1.407221
3,201810,TRAIN_000000,-0.057331,-0.057331,-0.057331,0.000000,0.0,0.0,0.0,0.222441,...,0.999998,0.595453,1.000911,0.999998,0.999998,-0.083067,0.044567,0.0,0.940814,1.587244
4,201811,TRAIN_000000,-0.036637,-0.036637,-0.036637,0.000000,0.0,0.0,0.0,0.244218,...,0.999998,0.593415,1.000995,0.999998,0.999998,-0.038451,0.082848,0.0,0.937722,1.478965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,201807,TRAIN_399999,1.999996,1.999996,1.999996,1.999996,0.0,0.0,0.0,2.001371,...,0.999998,0.765724,0.999998,0.999998,0.999998,0.066848,1.999996,0.0,0.000000,0.000000
1999996,201808,TRAIN_399999,0.048231,0.048231,0.062418,-1.999996,0.0,0.0,0.0,-1.999702,...,0.999998,0.774913,0.999998,0.999998,0.999998,0.002990,0.163849,0.0,0.000000,0.000000
1999997,201809,TRAIN_399999,0.063319,0.063319,0.063319,0.000000,0.0,0.0,0.0,0.287509,...,0.999998,0.770337,0.999998,0.999998,0.999998,-0.040103,0.085437,0.0,0.000000,0.000000
1999998,201810,TRAIN_399999,0.028755,0.028755,0.021574,-1.999996,0.0,0.0,0.0,-1.998637,...,0.999998,0.590594,0.999998,0.999998,0.999998,-0.118429,-0.047272,0.0,0.000000,0.000000


In [4]:
# segment 데이터 불러오기
segment = pd.read_parquet('open/train/1.회원정보/201807_train_회원정보.parquet')
segment_col = segment[['ID', 'Segment']]

In [5]:
# segment left outer join
df8 = df8.merge(segment_col, on='ID', how='left')
df8.head()

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,Segment
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.0,0.409766,...,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.0,1.044401,1.280543,D
1,201808,TRAIN_000000,-0.097118,-0.097118,-0.111664,-1.999996,0.0,0.0,0.0,0.192771,...,0.595262,1.000661,0.999998,0.999998,-0.140747,-0.117855,0.0,1.185349,1.555697,D
2,201809,TRAIN_000000,0.015812,0.015812,0.015812,0.000000,0.0,0.0,0.0,0.250439,...,1.048667,1.000722,0.999998,0.999998,-0.057002,0.052830,0.0,0.879170,1.407221,D
3,201810,TRAIN_000000,-0.057331,-0.057331,-0.057331,0.000000,0.0,0.0,0.0,0.222441,...,0.595453,1.000911,0.999998,0.999998,-0.083067,0.044567,0.0,0.940814,1.587244,D
4,201811,TRAIN_000000,-0.036637,-0.036637,-0.036637,0.000000,0.0,0.0,0.0,0.244218,...,0.593415,1.000995,0.999998,0.999998,-0.038451,0.082848,0.0,0.937722,1.478965,D


In [6]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 50 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   증감율_이용건수_신용_전월      float64
 3   증감율_이용건수_신판_전월      float64
 4   증감율_이용건수_일시불_전월     float64
 5   증감율_이용건수_할부_전월      float64
 6   증감율_이용건수_CA_전월      float64
 7   증감율_이용건수_체크_전월      float64
 8   증감율_이용건수_카드론_전월     float64
 9   증감율_이용금액_신용_전월      float64
 10  증감율_이용금액_신판_전월      float64
 11  증감율_이용금액_일시불_전월     float64
 12  증감율_이용금액_할부_전월      float64
 13  증감율_이용금액_CA_전월      float64
 14  증감율_이용금액_체크_전월      float64
 15  증감율_이용금액_카드론_전월     float64
 16  증감율_이용건수_신용_분기      float64
 17  증감율_이용건수_신판_분기      float64
 18  증감율_이용건수_일시불_분기     float64
 19  증감율_이용건수_할부_분기      float64
 20  증감율_이용건수_CA_분기      float64
 21  증감율_이용건수_체크_분기      float64
 22  증감율_이용건수_카드론_분기     float64
 23  증감율_이용금액_신용_분기      float64
 24  증감율_이용금액_신판_분기      floa

In [7]:
# 결측치 확인
print(df8.isnull().sum())

# 혜택수혜율_R3M: 402413
# 혜택수혜율_B0M: 461100

기준년월                       0
ID                         0
증감율_이용건수_신용_전월             0
증감율_이용건수_신판_전월             0
증감율_이용건수_일시불_전월            0
증감율_이용건수_할부_전월             0
증감율_이용건수_CA_전월             0
증감율_이용건수_체크_전월             0
증감율_이용건수_카드론_전월            0
증감율_이용금액_신용_전월             0
증감율_이용금액_신판_전월             0
증감율_이용금액_일시불_전월            0
증감율_이용금액_할부_전월             0
증감율_이용금액_CA_전월             0
증감율_이용금액_체크_전월             0
증감율_이용금액_카드론_전월            0
증감율_이용건수_신용_분기             0
증감율_이용건수_신판_분기             0
증감율_이용건수_일시불_분기            0
증감율_이용건수_할부_분기             0
증감율_이용건수_CA_분기             0
증감율_이용건수_체크_분기             0
증감율_이용건수_카드론_분기            0
증감율_이용금액_신용_분기             0
증감율_이용금액_신판_분기             0
증감율_이용금액_일시불_분기            0
증감율_이용금액_할부_분기             0
증감율_이용금액_CA_분기             0
증감율_이용금액_체크_분기             0
증감율_이용금액_카드론_분기            0
잔액_신판평균한도소진율_r6m           0
잔액_신판최대한도소진율_r6m           0
잔액_신판평균한도소진율_r3m           0
잔액_신판최대한도소진율_r3m           0
잔액_신판ca평균한도소진율

In [9]:

# 복사
df_corr = df8.copy()

# Segment 인코딩
le = LabelEncoder()
df_corr['Segment_enc'] = le.fit_transform(df_corr['Segment'])

# 숫자형 변수만 선택
numeric_cols = df_corr.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'Segment_enc' not in numeric_cols:
    numeric_cols.append('Segment_enc')

# 상관계수 계산 (변수 쌍별로 NaN 자동 제거)
corr = df_corr[numeric_cols].corr()

# Segment_enc 기준 상관계수 정렬
segment_corr = corr['Segment_enc'].sort_values(ascending=False)

# 보기 좋게 출력
pd.set_option('display.float_format', '{:.2f}'.format)
print(segment_corr)


Segment_enc           1.00
혜택수혜율_B0M             0.10
혜택수혜율_R3M             0.09
증감율_이용건수_할부_전월        0.08
증감율_이용건수_체크_전월        0.07
증감율_이용건수_CA_전월        0.06
증감율_이용건수_할부_분기        0.04
증감율_이용금액_CA_분기        0.03
증감율_이용금액_할부_분기        0.03
증감율_이용금액_체크_전월        0.03
증감율_이용건수_카드론_분기       0.02
증감율_이용금액_카드론_분기       0.02
증감율_이용금액_할부_전월        0.02
증감율_이용건수_CA_분기        0.02
증감율_이용금액_신용_분기        0.02
증감율_이용금액_신판_분기        0.02
변동률_잔액_CA_B1M         0.01
증감율_이용금액_CA_전월        0.01
증감율_이용금액_체크_분기        0.01
증감율_이용건수_체크_분기        0.01
증감율_이용금액_일시불_분기       0.01
증감율_이용금액_카드론_전월       0.01
증감율_이용건수_카드론_전월       0.01
증감율_이용건수_일시불_분기       0.00
증감율_이용건수_신판_분기        0.00
증감율_이용건수_신용_분기        0.00
기준년월                  0.00
변동률_잔액_B1M           -0.00
변동률_잔액_일시불_B1M       -0.00
증감율_이용건수_신용_전월       -0.01
증감율_이용건수_신판_전월       -0.01
증감율_이용건수_일시불_전월      -0.01
변동률_할부평잔             -0.02
변동률_CA평잔             -0.04
증감율_이용금액_신용_전월       -0.04
변동률_일시불평잔            -0.05
증감율_이용금액_신판_전월       -0.07
증

In [10]:
import pandas as pd

# 정렬된 결과를 딕셔너리 형태로 준비 (필요시 수정 가능)
segment_corr_ordered = {
    '기준년월': 0.00,
    'ID': None,
    '증감율_이용건수_신용_전월': -0.01,
    '증감율_이용건수_신판_전월': -0.01,
    '증감율_이용건수_일시불_전월': -0.01,
    '증감율_이용건수_할부_전월': 0.08,
    '증감율_이용건수_CA_전월': 0.06,
    '증감율_이용건수_체크_전월': 0.07,
    '증감율_이용건수_카드론_전월': 0.01,
    '증감율_이용금액_신용_전월': -0.04,
    '증감율_이용금액_신판_전월': -0.07,
    '증감율_이용금액_일시불_전월': -0.07,
    '증감율_이용금액_할부_전월': 0.02,
    '증감율_이용금액_CA_전월': 0.01,
    '증감율_이용금액_체크_전월': 0.03,
    '증감율_이용금액_카드론_전월': 0.01,
    '증감율_이용건수_신용_분기': 0.00,
    '증감율_이용건수_신판_분기': 0.00,
    '증감율_이용건수_일시불_분기': 0.00,
    '증감율_이용건수_할부_분기': 0.04,
    '증감율_이용건수_CA_분기': 0.02,
    '증감율_이용건수_체크_분기': 0.01,
    '증감율_이용건수_카드론_분기': 0.02,
    '증감율_이용금액_신용_분기': 0.02,
    '증감율_이용금액_신판_분기': 0.02,
    '증감율_이용금액_일시불_분기': 0.01,
    '증감율_이용금액_할부_분기': 0.03,
    '증감율_이용금액_CA_분기': 0.03,
    '증감율_이용금액_체크_분기': 0.01,
    '증감율_이용금액_카드론_분기': 0.02,
    '잔액_신판평균한도소진율_r6m': -0.18,
    '잔액_신판최대한도소진율_r6m': -0.20,
    '잔액_신판평균한도소진율_r3m': -0.19,
    '잔액_신판최대한도소진율_r3m': -0.20,
    '잔액_신판ca평균한도소진율_r6m': -0.25,
    '잔액_신판ca최대한도소진율_r6m': -0.27,
    '잔액_신판ca평균한도소진율_r3m': -0.26,
    '잔액_신판ca최대한도소진율_r3m': -0.26,
    '변동률_일시불평잔': -0.05,
    '변동률_RV일시불평잔': -0.10,
    '변동률_할부평잔': -0.02,
    '변동률_CA평잔': -0.04,
    '변동률_RVCA평잔': -0.10,
    '변동률_카드론평잔': -0.07,
    '변동률_잔액_B1M': -0.00,
    '변동률_잔액_일시불_B1M': -0.00,
    '변동률_잔액_CA_B1M': 0.01,
    '혜택수혜율_R3M': 0.09,
    '혜택수혜율_B0M': 0.10,
}

# DataFrame 변환
df_segment_corr = pd.DataFrame(
    list(segment_corr_ordered.items()),
    columns=['변수명', 'Segment와의 상관계수']
)

# Excel 저장
df_segment_corr.to_excel('04.xlsx', index=False)
